In [1]:
import os
%pwd

'd:\\Sentiment Anaylsis\\research'

In [2]:
os.chdir("../")

In [5]:
import os
from box.exceptions import BoxValueError
import yaml

import json
import joblib
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any
import base64
from pathlib import Path

In [3]:
def create_directories(path_to_dir:list,verbose=True):
    for path in path_to_dir:
        os.makedirs(path,exist_ok=True)
       # if verbose:
           # logger.info(f"created dierctor ast {path}")


In [6]:
def read_yaml(path_to_yaml:Path)-> ConfigBox:
    try:
        with open(path_to_yaml) as yaml_file:
            content=yaml.safe_load(yaml_file)
           # logger.info(f"yaml file {path_to_yaml} load sucessfuly")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise(e)

In [50]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class Datatransformationconfig:
    root_dir:Path
    source_name: str
    local_file: str
    transformed_path: Path
    

In [51]:
class ConfigurationManger:
    def __init__(self,config_file_path,params_file_path):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> Datatransformationconfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config=Datatransformationconfig(


            root_dir=config.root_dir,


            
            source_name= config.source_name,
           
            local_file=config.local_file,
            transformed_path=config.transformed_path

                
        )
      
        return data_transformation_config
    
        

In [69]:
from datasets import load_from_disk
from transformers import AutoTokenizer

class Datatransformation:
    def __init__(self, config:Datatransformationconfig):
        self.config=config 
    
        self.dataset=load_from_disk(self.config.local_file)
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.source_name)
        self.tokenized=None

    def convert_examples_to_features(self, example):
        input_encodings = self.tokenizer(example['review'], max_length=128, truncation=True, padding='max_length')
        labels = [rating - 1 for rating in example['star']]

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': labels
        }
    def transformed(self):
        tokenized_datasets = self.dataset.map(self.convert_examples_to_features, batched=True)
        tokenized_datasets.save_to_disk(self.config.transformed_path)




In [32]:
dataset=load_from_disk('artifacts/save_dataset')

In [70]:
try:
    config=ConfigurationManger(config_file_path=Path("config.yaml"),params_file_path=Path("params.yaml"))
    datatransformation_config=config.get_data_transformation_config()
    data_transformation=Datatransformation(config=datatransformation_config)

    
    data_transformation.transformed()
except Exception as e:
    raise e

Saving the dataset (1/1 shards): 100%|██████████| 288065/288065 [00:00<00:00, 820632.62 examples/s]
